In [1]:
# ライブラリimport
import argparse
import logging
import os

import pandas as pd
from pandas import DataFrame
import pandas.tseries.offsets as offsets
import numpy as np
from datetime import date, timedelta, time, datetime
from tabulate import tabulate

from itertools import product

from ailab_tools.smn import ImpalaResource
from ailab_tools.utils import GMail, GSheet

from collections import OrderedDict

import zipfile
import glob

import gc

from ailab_tools.magics import slack_notify

In [2]:
today_yymmdd = date.today().strftime("%y%m%d")
print(today_yymmdd)

190805


In [3]:
#クロスデバイステーブル
xd_table = 'segment.drawbridge_device'

3992	株式会社 新学社

In [4]:
#デバイス
seed_sp_device = 'iOS','Android'
xd_sp_device = 'iphone','android_phone','windows_phone'
xd_pc_device = 'windows_computer','linux_computer','mac','android_tablet','ipad','windows_tablet'

#ターゲット
advertiser_id = 3992
retargeting_group_id = 111776 #XD_00_CV

grp = 'rtg'

In [5]:
uid_table = f"dm_tmp.ya_xd_advid_{advertiser_id}_{grp}_{retargeting_group_id}_{today_yymmdd}"
print(uid_table)

dm_tmp.ya_xd_advid_3992_rtg_111776_190805


SP⇔PC

In [6]:
%%slack_notify @yusuke_akada

#f-string
query = (
    f"""
    select 
    distinct 
    db2.smn_uid
    ,case when db2.device in ('iphone','android_phone','windows_phone') then 'SP' ELSE 'PC' END as xd_device
    from dsplog.adserver ads
    inner join segment.drawbridge_device db
    on ads.smn_uid = db.smn_uid
    inner join segment.drawbridge_device db2
    on db.drawbridge_consumer_id = db2.drawbridge_consumer_id
    where concat_ws('-',year,month,day) between to_date(days_add(now(),-30)) and to_date(days_add(now(),-1))
    and advertiser_id = {advertiser_id}
    and retargeting_group_id = {retargeting_group_id}
/*  and ads.user_agent_os_family in {seed_sp_device}*/
/*  and db2.device in {xd_pc_device}*/
    and db2.smn_uid > 0
    and ads.smn_uid <> db2.smn_uid
    limit 10000
    """
    )

with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        df = ir.sql_to_pandas(query)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    s

In [7]:
df.shape[0]

89

In [8]:
df.groupby(["xd_device"])["smn_uid"].count()

xd_device
PC    27
SP    62
Name: smn_uid, dtype: int64

In [9]:
df_fix = df.drop("xd_device", axis=1)

In [10]:
df_fix.to_csv(f'{uid_table}.tsv', sep='\t', index=False, header=False)

3992	株式会社 新学社

In [11]:
#デバイス
seed_sp_device = 'iOS','Android'
xd_sp_device = 'iphone','android_phone','windows_phone'
xd_pc_device = 'windows_computer','linux_computer','mac','android_tablet','ipad','windows_tablet'

#ターゲット
advertiser_id = 3992
retargeting_group_id = 111777 # XD_11_フォーム（お試し）_LP

grp = 'rtg'

In [12]:
uid_table = f"dm_tmp.ya_xd_advid_{advertiser_id}_{grp}_{retargeting_group_id}_{today_yymmdd}"
print(uid_table)

dm_tmp.ya_xd_advid_3992_rtg_111777_190805


SP⇔PC

In [13]:
%%slack_notify @yusuke_akada

#f-string
query = (
    f"""
    select 
    distinct 
    db2.smn_uid
    ,case when db2.device in ('iphone','android_phone','windows_phone') then 'SP' ELSE 'PC' END as xd_device
    from dsplog.adserver ads
    inner join segment.drawbridge_device db
    on ads.smn_uid = db.smn_uid
    inner join segment.drawbridge_device db2
    on db.drawbridge_consumer_id = db2.drawbridge_consumer_id
    where concat_ws('-',year,month,day) between to_date(days_add(now(),-30)) and to_date(days_add(now(),-1))
    and advertiser_id = {advertiser_id}
    and retargeting_group_id = {retargeting_group_id}
/*  and ads.user_agent_os_family in {seed_sp_device}*/
/*  and db2.device in {xd_pc_device}*/
    and db2.smn_uid > 0
    and ads.smn_uid <> db2.smn_uid
    limit 10000
    """
    )

with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        df = ir.sql_to_pandas(query)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    s

In [14]:
df.shape[0]

4571

In [15]:
df.groupby(["xd_device"])["smn_uid"].count()

xd_device
PC    1418
SP    3153
Name: smn_uid, dtype: int64

In [16]:
df_fix = df.drop("xd_device", axis=1)

In [17]:
df_fix.to_csv(f'{uid_table}.tsv', sep='\t', index=False, header=False)

3992	株式会社 新学社

In [18]:
#デバイス
seed_sp_device = 'iOS','Android'
xd_sp_device = 'iphone','android_phone','windows_phone'
xd_pc_device = 'windows_computer','linux_computer','mac','android_tablet','ipad','windows_tablet'

#ターゲット
advertiser_id = 3992
retargeting_group_id = 111779 # XD_12_フォーム（入会）

grp = 'rtg'

In [25]:
uid_table = f"dm_tmp.ya_xd_advid_{advertiser_id}_{grp}_{retargeting_group_id}_{today_yymmdd}"
print(uid_table)

dm_tmp.ya_xd_advid_3992_rtg_111779_190805


SP⇔PC

In [20]:
%%slack_notify @yusuke_akada

#f-string
query = (
    f"""
    select 
    distinct 
    db2.smn_uid
    ,case when db2.device in ('iphone','android_phone','windows_phone') then 'SP' ELSE 'PC' END as xd_device
    from dsplog.adserver ads
    inner join segment.drawbridge_device db
    on ads.smn_uid = db.smn_uid
    inner join segment.drawbridge_device db2
    on db.drawbridge_consumer_id = db2.drawbridge_consumer_id
    where concat_ws('-',year,month,day) between to_date(days_add(now(),-30)) and to_date(days_add(now(),-1))
    and advertiser_id = {advertiser_id}
    and retargeting_group_id = {retargeting_group_id}
/*  and ads.user_agent_os_family in {seed_sp_device}*/
/*  and db2.device in {xd_pc_device}*/
    and db2.smn_uid > 0
    and ads.smn_uid <> db2.smn_uid
    limit 10000
    """
    )

with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        df = ir.sql_to_pandas(query)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    s

In [26]:
df.shape[0]

1677

In [27]:
df.groupby(["xd_device"])["smn_uid"].count()

xd_device
PC     509
SP    1168
Name: smn_uid, dtype: int64

In [28]:
df_fix = df.drop("xd_device", axis=1)

In [29]:
df_fix.to_csv(f'{uid_table}.tsv', sep='\t', index=False, header=False)

PC→SP

In [ ]:
%%slack_notify @yusuke_akada

#f-string
query = (
    f"""
    select 
    distinct 
    db2.smn_uid
    ,case when db2.device in ('iphone','android_phone','windows_phone') then 'SP' ELSE 'PC' END as xd_device
    from dsplog.adserver ads
    inner join segment.drawbridge_device db
    on ads.smn_uid = db.smn_uid
    inner join segment.drawbridge_device db2
    on db.drawbridge_consumer_id = db2.drawbridge_consumer_id
    where concat_ws('-',year,month,day) between to_date(days_add(now(),-30)) and to_date(days_add(now(),-1))
    and advertiser_id = {advertiser_id}
    and conversion_group_id = {conversion_group_id}
    and ads.user_agent_os_family not in {seed_sp_device}
    and db2.device in {xd_sp_device}
    and db2.smn_uid > 0
    and ads.smn_uid <> db2.smn_uid
    limit 10000
    """
    )

with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        df = ir.sql_to_pandas(query)

In [ ]:
df.shape[0]

In [ ]:
df.groupby(["xd_device"])["smn_uid"].count()

In [ ]:
df_fix = df.drop("xd_device", axis=1)

In [ ]:
df_fix.to_csv(f'{uid_table}.tsv', sep='\t', index=False, header=False)

#複数回接触パターン

%%slack_notify @yusuke_akada

#f-string
query = (
    f"""
    with tmp as (
    select smn_uid,count(1) as act_count
    from dsplog.adserver
    where concat_ws('-',year,month,day) BETWEEN to_date(days_add(now(),-30)) and to_date(days_add(now(),-1))
    and retargeting_group_id = {retargeting_group_id}
    and advertiser_id = {advertiser_id}
    group by smn_uid
    )
    ,tmp2 as (
    select smn_uid from tmp where act_count >= 3 group by smn_uid
    )
    select distinct db2.smn_uid 
    from tmp2 ads
    inner join segment.drawbridge_device db
    on ads.smn_uid = db.smn_uid
    inner join segment.drawbridge_device db2
    on db.drawbridge_consumer_id = db2.drawbridge_consumer_id
    and db2.smn_uid > 0
    limit 10000
    """
    )

with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        df = ir.sql_to_pandas(query)

In [ ]:
df.shape[0]

In [ ]:
df.to_csv(f'{uid_table}.tsv', sep='\t', index=False, header=False)

In [ ]:
#デバイス
seed_sp_device = 'iOS','Android'
xd_sp_device = 'iphone','android_phone','windows_phone'

#ターゲット
advertiser_id = 8425
conversion_group_id = 14376 #CV（無料体験）    

grp = 'cv'

In [ ]:
uid_table = f"dm_tmp.ya_xd_advid_{advertiser_id}_{grp}_{conversion_group_id}_{today_yymmdd}"
print(uid_table)